In [1]:

%pprint
import sys
if (osp.join('..', 'py') not in sys.path): sys.path.insert(1, osp.join('..', 'py'))

Pretty printing has been turned OFF


In [2]:

from FRVRS import (nu, fu, display, isna)
import os
import pandas as pd

In [3]:

# Get all CSVs into one data frame
frvrs_logs_df = nu.load_data_frames(frvrs_logs_df='frvrs_logs_df')['frvrs_logs_df']
print(frvrs_logs_df.shape) # (829116, 113)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl.
Argument 'placement' has incorrect type (expected pandas._libs.internals.BlockPlacement, got slice)
No pickle exists for frvrs_logs_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv.
(829116, 113)



# Replace the tool applied sender missing patient ID

In [4]:

match_series = (frvrs_logs_df.tool_applied_sender == 'Needle_Decomp(Clone) (UnityEngine.GameObject)') & frvrs_logs_df.tool_applied_patient_id.isnull()
df = frvrs_logs_df[match_series]
print(df.shape)
df.sample(6).dropna(axis='columns', how='all').T

(634, 113)


,708726,751501,690065,44341,640191,3463
action_type,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED
action_tick,131137,218433,828907,100298,510763,127153
event_time,2023-03-03 12:15:00,2023-06-23 08:57:00,2023-03-03 10:36:00,2023-03-14 14:37:55,2022-03-15 11:14:00,2023-03-10 08:57:12
session_uuid,206da802-d38f-4f3a-98e3-44f99481479d,93d6665a-9210-4bb9-b084-69d6e7335d04,e662a811-40fd-4bf2-b9df-d2d221d07b4a,f02dc87e-8f04-4ddd-90fd-74dce1206bd6,22e2c9a3-b93f-4f7f-896a-ca188c78505b,50e4351a-3afd-43ff-a65f-f84eaab0d0fd
file_name,v.1.0/Clean Marty7.csv,v.1.0/clean-93d6665a-9210-4bb9-b084-69d6e7335d...,v.1.0/Clean Marty1.csv,All CSV files renamed by date/03.14.23.1448.csv,Double runs removed/22.03.15.1106r.csv,All CSV files renamed by date/03.10.23.0855.csv
logger_version,1.0,1.0,1.0,1.3,1.0,1.3
tool_applied_type,Needle,Needle,Needle,Needle,Needle,Needle
tool_applied_attachment_point,RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject)
tool_applied_tool_location,cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject)
tool_applied_data,right_chest,right_chest,right_chest,right_chest,right_chest,right_chest


In [10]:

match_series = (frvrs_logs_df.tool_applied_sender == 'Needle_Decomp(Clone) (UnityEngine.GameObject)') & frvrs_logs_df.tool_applied_patient_id.isnull()
for (session_uuid, scene_id), tool_applied_df in frvrs_logs_df[match_series].groupby(fu.scene_groupby_columns):
    
    # Get the elapsed time history of this scene
    tool_applied_mask_series = True
    for cn in fu.scene_groupby_columns: tool_applied_mask_series &= (frvrs_logs_df[cn] == eval(cn))
    action_tick = tool_applied_df.action_tick.min()
    tool_applied_mask_series &= (frvrs_logs_df.action_tick == action_tick)
    scene_df = frvrs_logs_df[tool_applied_mask_series]
    # display(scene_df.sample(min(6, scene_df.shape[0])).dropna(axis='columns', how='all').T); break

    # Set patient ID to the first valid instance
    patient_id = None
    srs = scene_df.patient_id
    first_valid_index = srs.first_valid_index()
    if first_valid_index is None:
        
        # Get the entire history of this scene
        mask_series = True
        for cn in fu.scene_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
        scene_df = frvrs_logs_df[mask_series]
        
        # Get the index of the row where the action_tick equals the needle action_tick
        mask_series = (scene_df.action_tick <= action_tick)
        df = scene_df[mask_series].sort_values('action_tick', ascending=False)
        
        # Iterate backwards through the scene data frame from the row_index until a non-null value is found in the patient ID column
        for i in df.index:
            if not isna(scene_df.loc[i, 'patient_id']):
                patient_id = scene_df.loc[i, 'patient_id']
                break
        
    else: patient_id = srs[first_valid_index]
    if patient_id is not None:
        frvrs_logs_df.loc[tool_applied_mask_series, 'patient_id'] = patient_id
        tool_applied_mask_series &= (frvrs_logs_df.action_type == 'TOOL_APPLIED')
        frvrs_logs_df.loc[tool_applied_mask_series, 'tool_applied_patient_id'] = patient_id

In [11]:

match_series = (frvrs_logs_df.tool_applied_sender == 'Needle_Decomp(Clone) (UnityEngine.GameObject)') & frvrs_logs_df.tool_applied_patient_id.isnull()
df = frvrs_logs_df[match_series]
print(df.shape) # (25, 113)
df.sample(min(6, df.shape[0])).dropna(axis='columns', how='all').T

(0, 113)


""


In [12]:

nu.store_objects(frvrs_logs_df=frvrs_logs_df)
nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
print(frvrs_logs_df.shape) # (829277, 113)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 113)
